# Data Preprocessing

23450320 XIAN Yilang, *DAAI*

In [320]:
import zipfile

zip_file = zipfile.ZipFile('archive.zip')
zip_file.extractall()
zip_file.close()

In [321]:
import pandas as pd

df = pd.read_csv('steam_games.csv')

In [322]:
# Drop some useless columns

useless_columns = ['types', 'desc_snippet', 'recent_reviews', 'languages', 'achievements', 'mature_content', 'minimum_requirements', 'recommended_requirements']

df = df.drop(useless_columns, axis=1)

In [323]:
# Drop some rows with null values

columns_should_not_be_null = ['all_reviews', 'name', 'popular_tags', 'game_details', 'genre', 'game_description']

for column in columns_should_not_be_null:
    df = df.drop(index=df[pd.isna(df[column])].index, axis=0)

In [324]:
# Merge "original_price" and "discount_price" into "price"

for index in df[pd.isna(df['original_price'])].index:
    df.loc[index, 'original_price'] = df.loc[index, 'discount_price']

df = df.drop(index=df[pd.isna(df['original_price'])].index, axis=0)

df = df.drop('discount_price', axis=1)

df = df.rename(columns={'original_price': 'price'})

In [325]:
# Drop some rows with null values again

columns_should_not_be_null = ['release_date', 'developer', 'publisher']

for column in columns_should_not_be_null:
    df = df.drop(index=df[pd.isna(df[column])].index, axis=0)

In [326]:
# Create column "all_reviews_simple"

review_levels = ['Overwhelmingly Negative', 'Very Negative', 'Negative', 'Mostly Negative', 'Mixed', 'Mostly Positive', 'Positive', 'Very Positive', 'Overwhelmingly Positive']

simple_reviews = list()
for index in df.index:
    simple_review = df.loc[index, 'all_reviews'].split(',')[0]
    if simple_review not in review_levels:
        simple_review = "Too Few Reviews"
    simple_reviews.append(simple_review)
df.insert(loc=3, column='all_reviews_simple', value=simple_reviews)

In [327]:
# Create column "all_reviews_simple"

# Steam Review Explanation:
# Overwhelmingly Positive (95 - 99%)
# Very Positive (80 - 94% or 95 - 99% with moderate reviews)
# Positive (80 - 99% with few reviews)
# Mostly Positive (70 - 79%)
# Mixed (40 - 69%)
# Mostly Negative (20 - 39%)
# Negative (0 - 19% with few reviews)
# Very Negative (0 - 19%)
# Overwhelmingly Negative (0 - 19% with moderate reviews)
# Too Few Reviews (less than 10 reviews)

review_score = {'Too Few Reviews': 3,
                'Overwhelmingly Negative': 1,
                'Very Negative': 1,
                'Negative': 2,
                'Mostly Negative': 2,
                'Mixed': 3,
                'Mostly Positive': 4,
                'Positive': 4,
                'Very Positive': 5,
                'Overwhelmingly Positive': 5}

review_scores = list()
for index in df.index:
    review_scores.append(review_score[df.loc[index, 'all_reviews_simple']])
df.insert(loc=4, column='all_reviews_score', value=review_scores)

In [328]:
# Delete "About This Game" segment in colomn "game_description"

for index in df.index:
    description = df.loc[index, 'game_description']
    description = description.replace('About This Game', '')
    description = description.strip()
    df.loc[index, 'game_description'] = description

In [329]:
# Normalize column "price" and create column "price_numeric"

# import re

price = df['price'].copy(deep=True)
for index in df.index:
    if df.loc[index, 'price'][0] == '$':
        price[index] = price[index][1:]
    else:
        df.loc[index, 'price'] = "Free to Play"
        price[index] = 0.00
    # elif not bool(re.match(pattern=r'^\d+(\.\d+)?$', string=df.loc[index, 'price'])):
    #    df.loc[index, 'price'] = 0.00
price = price.astype(pd.Float32Dtype())
df.insert(loc=13, column='price_numeric', value=price)

In [330]:
# "popular_tags" maybe too many

df = df.drop('popular_tags', axis=1)

In [331]:
# Clean "game_details"

import copy

useless_game_details = [
    'Captions available',
    'Commentary available',
    'Downloadable Content',
    'Game demo',
    'In-App Purchases',
    'Includes Source SDK',
    'Mods',
    'Mods (require HL2)',
    'Partial Controller Support',
    'Profile Features Limited\xa0\r\n\t\t\t\t\t\t\t\t\t',
    'Stats',
    'Steam Achievements',
    'Steam Cloud',
    'Steam Leaderboards',
    'Steam Trading Cards',
    'Steam Turn Notifications',
    'Steam Workshop',
    'Steam is learning about this downloadable content\xa0\r\n\t\t\t\t\t\t\t\t\t',
    'Steam is learning about this game\xa0\r\n\t\t\t\t\t\t\t\t\t',
    'Steam is learning about this software\xa0\r\n\t\t\t\t\t\t\t\t\t',
    'SteamVR Collectibles',
    'Valve Anti-Cheat enabled',
    'MMO'
]

for index in df.index:
    game_details_list = df.loc[index, 'game_details'].split(',')
    game_details_str = ''
    for game_detail in game_details_list:
        if game_detail not in useless_game_details:
            game_details_str = game_details_str + ',' + game_detail
    game_details_str = game_details_str[1:]
    df.loc[index, 'game_details'] = copy.deepcopy(game_details_str)

In [332]:
# Clean "genre"

import copy

useless_genre = [
    'Accounting',
    'Animation & Modeling',
    'Audio Production',
    'Design & Illustration',
    'Early Access',
    'Education',
    'Free to Play',
    'Game Development',
    'Movie',
    'Photo Editing',
    'Software Training',
    'Utilities',
    'Video Production',
    'Web Publishing'
]

for index in df.index:
    genre_list = df.loc[index, 'genre'].split(',')
    genre_str = ''
    for genre in genre_list:
        if genre not in useless_genre:
            genre_str = genre_str + ',' + genre
    genre_str = genre_str[1:]
    df.loc[index, 'genre'] = copy.deepcopy(genre_str)

In [333]:
# Merge "game_details" and "genre" into "genre"
import copy

for index in df.index:
    if df.loc[index, 'game_details'] == '':
        if df.loc[index, 'genre'] == '':
            df = df.drop(index, axis=0)
            continue
        continue
    if df.loc[index, 'genre'] == '':
        df.loc[index, 'genre'] = copy.deepcopy(df.loc[index, 'game_details'])
        continue
    df.loc[index, 'genre'] = df.loc[index, 'game_details'] + ',' + df.loc[index, 'genre']

df = df.drop('game_details', axis=1)

In [334]:
# Drop some games whose "genre == ''" or "game_description == ''"

for index in df.index:
    if (df.loc[index, 'genre'] == '') or (df.loc[index, 'game_description'] == ''):
        df = df.drop(index, axis=0)

In [335]:
# Reset the DataFrame index

df = df.reset_index(drop=True)

In [336]:
# Drop Batman: Arkham City - Game of the Year Edition for its abnormal url

df = df.drop(13697, axis=0)
df = df.reset_index(drop=True)

In [337]:
# Create game_id from url

game_id = [] 

for index in df.index:

    game_id.append(int(df.loc[index, 'url'].split('/')[-3]))
    if not df.loc[index, 'url'].split('/')[-3].isdigit():
        print(index)
        print(df.loc[index], 'name')

df.insert(loc=0, column='game_id', value=game_id)

In [338]:
# Generate One-Hot encoding from column "genre"

steam_features_source_columns = ['genre']

def get_features_string(index):
    features = ''
    for column in steam_features_source_columns:
        features = features + ',' + df.loc[index, column]
    features = features[1:]
    return features

def get_row_dict(features):
    dic = dict()
    for feature in features:
        dic.update({feature: int(1)})
    return dic

steam_features = set()
for column in steam_features_source_columns:
    for index in df.index:
        features = df.loc[index, column].split(',')
        steam_features.update(features)
steam_features = list(steam_features)

rows = []
for index in df.index:
    features_string = get_features_string(index)
    features_list = features_string.split(',')
    rows.append(get_row_dict(features_list))

one_hot = pd.DataFrame(rows, columns=steam_features)
one_hot = one_hot.fillna(0)
one_hot = one_hot.astype('int8')

In [339]:
df = df.join(one_hot)

In [352]:
# Random check

import random

def random_check(index=None):
    if not index:
        index = random.randint(0, len(df) - 1)
    print(index)
    features_string = get_features_string(index)

    print(df.loc[index, 'name'])
    print(features_string)

    features_list = features_string.split(',')
    for feature in features_list:
        print(f"{feature} == 1 is {bool(df.loc[index, feature])}")

random_check()

8979
HotFloor
Single-player,Action
Single-player == 1 is True
Action == 1 is True


In [353]:
df.to_csv("steam_games_with_one_hot.csv", index=False)


In [354]:
df.to_excel("steam_games_with_one_hot.xlsx", index=False)